<a href="https://colab.research.google.com/github/chu-ise/378A-2022/blob/main/notebooks/03/02_wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Environment setup to persist data in Google Drive

In [1]:
import os, sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    if not os.path.exists('/content/drive'):
      drive.mount("/content/drive")
      !ln -s "/content/drive/My Drive/colab_workspace" workspace
    WORKSPACE_DIR = "/content/workspace"
else:
    WORKSPACE_DIR = "../../workspace" 
print(f'WORKSPACE_DIR = {WORKSPACE_DIR}')
data_dir = os.path.join(WORKSPACE_DIR, "data")
os.makedirs(data_dir, exist_ok=True)

Mounted at /content/drive
WORKSPACE_DIR = /content/workspace


In [2]:
import gdown, os
id = "142Cd7boufftU56nIgbOnqerxWcgQblsO"

data_file = "conf.zip"
conf_dir = data_file.split(".")[0]
if not os.path.exists(conf_dir):
    gdown.download(id=id, output=data_file, quiet=False)
    !unzip $data_file
    os.remove(data_file)
else:
    print(f"{conf_dir} already exists")

Downloading...
From: https://drive.google.com/uc?id=142Cd7boufftU56nIgbOnqerxWcgQblsO
To: /content/conf.zip
100%|██████████| 1.86k/1.86k [00:00<00:00, 1.56MB/s]

Archive:  conf.zip
   creating: conf/
  inflating: conf/config.yaml        
   creating: conf/downloader/
  inflating: conf/downloader/kowiki.yaml  
  inflating: conf/downloader/bok.yaml  
  inflating: conf/downloader/_default.yaml  
  inflating: conf/downloader/enwiki.yaml  


In [3]:
%%capture
%pip install ekorpkit[dataset]
%pip install ekorpkit[wiki]

### Download and Extract Wiki Dump

In [4]:
from ekorpkit.io.fetch.loader.wiki import Wiki

In [5]:
import os
from hydra import initialize, compose

In [6]:
with initialize(config_path="conf"):
    cfg=compose(
        overrides=[
            "+downloader=kowiki",
            f"+data_dir={data_dir}",
        ]
    )
    print(cfg)

{'downloader': {'name': 'kowiki', 'output_dir': '${..data_dir}/${.name}', 'output_file': None, 'autoload': False, 'force_download': False, 'num_workers': 1, 'compress': False, 'verbose': True, '_target_': 'ekorpkit.io.fetch.loader.wiki.Wiki', 'extract': {'_target_': 'extract_wiki'}, 'dump': {'_target_': 'web_download', 'url': 'https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles.xml.bz2', 'dump_dir': '${..output_dir}/dump', 'dump_file': '${..name}.xml.bz2'}}, 'data_dir': '/content/workspace/data'}


In [7]:
wk = Wiki(**cfg.downloader)

In [8]:
wk.download_dump()

[kowiki] download kowiki.xml.bz2: 820MB [02:46, 4.91MB/s]                           


In [10]:
# wk.extract_wiki()

In [9]:
wk.fetch??

In [16]:
from ekorpkit.io.parse.json import load_jsonlines

wiki_sample_file = '/content/workspace/data/kowiki/wiki_00'
with open(wiki_sample_file) as fo:
  wiki_parsed = load_jsonlines(fo.read())

In [21]:
wiki_parsed[0]

{'id': '5',
 'revid': '710721',
 'text': '제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이며, 독재자의 사신이라는 별명을 가지고 있다.\n생애.\n어린 시절.\n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n정계 입문.\n1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n대통령 재임.\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받고 제럴드 포드 대통령을 누르고 당선되었다.\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.\n외교 정책.\n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다.\n임기 말, 소련의 아프가니스탄 침공 사건으로 인해 1980년 하계 올림픽에 반공국가들의 보이콧을 

In [23]:
import pandas as pd

df = pd.DataFrame(wiki_parsed)
df.head()

,id,revid,url,title,text
0,5,710721,https://ko.wikipedia.org/wiki?curid=5,지미 카터,"제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대..."
1,9,123884,https://ko.wikipedia.org/wiki?curid=9,수학,"수학(數學, ,줄여서 math)은 수, 양, 구조, 공간, 변화 등의 개념을 다루는..."
2,10,368112,https://ko.wikipedia.org/wiki?curid=10,수학 상수,"수학에서 상수란 그 값이 변하지 않는 불변량으로, 변수의 반대말이다. 물리 상수와는..."
3,19,368112,https://ko.wikipedia.org/wiki?curid=19,문학,"문학(文學, )은 언어를 예술적 표현의 제재로 삼아 새로운 의미를 창출하여, 인간과..."
4,20,368112,https://ko.wikipedia.org/wiki?curid=20,나라 목록,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...
